In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
from tqdm import tqdm

In [20]:
def em_and_f1(p_url, acc, id="L1", rounded_to_num=5):

    page = requests.get(p_url)
    soup = BeautifulSoup(page.content)

    tr = soup.find('tr', attrs={'id': id})
    td = tr.find(attrs={'class': 'overflow-visible whitespace-pre px-3'}).text
    x = re.findall("\d+\.\d+", td)

    if acc == True: return round(float(x[0]), rounded_to_num)
    else: return round(float(x[1]), rounded_to_num)

# Intermediate task transfer-learning

In [21]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-"
suffix = "-LR-1e-05/blob/main/results/evaluation/metric_result.txt"

In [22]:
def take_all_url_ittl(data):

    ittl_arr = ["without-ITTL", "with-ITTL"]
    freeze_arr = ["without-freeze", "with-freeze"]

    url_arr = []
    for ittl in ittl_arr:
        for freeze in freeze_arr:

            if ittl == "without-ITTL":
                freeze = "without-freeze"
            
            url_arr.append(f"{prefix}{data}-with-xlm-roberta-large-{ittl}-{freeze}{suffix}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [23]:
def extract_values_ittl(url, data):

    ittl_pattern = re.compile(r'with-ITTL')
    freeze_pattern = re.compile(r'with-freeze')

    ittl_match = ittl_pattern.search(url)
    ittl = bool(ittl_match)

    freeze_match = freeze_pattern.search(url)
    freeze = bool(freeze_match)

    EM = em_and_f1(url, True)
    F1 = em_and_f1(url, False)

    if data == "IDK-MRC":
        data = "idkmrc"
    elif data == "TYDI-QA-ID":
        data = "tydiqaid"
    elif data == "Squad-ID":
        data = "squadid"

    return data, ittl, freeze, EM, F1

In [24]:
def create_table_ittl(url_list, data):
    
    data_list = []
    for url in tqdm(url_list, desc="Processing URLs", unit="URL"):
        extracted_values = extract_values_ittl(url, data)
        data_list.append(extracted_values)

    df = pd.DataFrame(data_list, columns=['data', 'ITTL', 'Freeze', 'EM', 'F1'])
    
    df.loc[0, 'ITTL'] = "Baseline"
    df.loc[0, 'Freeze'] = "Baseline"
    
    return df

## IDK-MRC

In [25]:
data = "IDK-MRC"
url_list = take_all_url_ittl(data)
table = pd.DataFrame(create_table_ittl(url_list, data))
table.to_excel(f'{data}-ittl.xlsx', index=False)
table

Amount of url: 3


Processing URLs: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.10s/URL]


,data,ITTL,Freeze,EM,F1
0,idkmrc,Baseline,Baseline,77.94811,84.63667
1,idkmrc,True,False,77.94811,84.23181
2,idkmrc,True,True,78.30189,85.14123


In [26]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

IDK-MRC sorted by EM


,data,ITTL,Freeze,EM,F1
2,idkmrc,True,True,78.30189,85.14123
0,idkmrc,Baseline,Baseline,77.94811,84.63667
1,idkmrc,True,False,77.94811,84.23181


In [27]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

IDK-MRC sorted by F1


,data,ITTL,Freeze,EM,F1
2,idkmrc,True,True,78.30189,85.14123
0,idkmrc,Baseline,Baseline,77.94811,84.63667
1,idkmrc,True,False,77.94811,84.23181


## TYDI-QA-ID

In [28]:
data = "TYDI-QA-ID"
url_list = take_all_url_ittl(data)
table = pd.DataFrame(create_table_ittl(url_list, data))
table.to_excel(f'{data}-ittl.xlsx', index=False)
table

Amount of url: 3


Processing URLs: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.20URL/s]


,data,ITTL,Freeze,EM,F1
0,tydiqaid,Baseline,Baseline,66.51109,78.26768
1,tydiqaid,True,False,65.11085,76.39569
2,tydiqaid,True,True,66.51109,77.58959


In [29]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

TYDI-QA-ID sorted by EM


,data,ITTL,Freeze,EM,F1
0,tydiqaid,Baseline,Baseline,66.51109,78.26768
2,tydiqaid,True,True,66.51109,77.58959
1,tydiqaid,True,False,65.11085,76.39569


In [30]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

TYDI-QA-ID sorted by F1


,data,ITTL,Freeze,EM,F1
0,tydiqaid,Baseline,Baseline,66.51109,78.26768
2,tydiqaid,True,True,66.51109,77.58959
1,tydiqaid,True,False,65.11085,76.39569


## SQUAD-ID

In [31]:
data = "Squad-ID"
url_list = take_all_url_ittl(data)
table = pd.DataFrame(create_table_ittl(url_list, data))
table.to_excel(f'{data}-ittl.xlsx', index=False)
table

Amount of url: 3


Processing URLs: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.19URL/s]


,data,ITTL,Freeze,EM,F1
0,squadid,Baseline,Baseline,49.54021,66.69283
1,squadid,True,False,49.94148,67.20374
2,squadid,True,True,50.40963,67.63862


In [32]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

Squad-ID sorted by EM


,data,ITTL,Freeze,EM,F1
2,squadid,True,True,50.40963,67.63862
1,squadid,True,False,49.94148,67.20374
0,squadid,Baseline,Baseline,49.54021,66.69283


In [33]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

Squad-ID sorted by F1


,data,ITTL,Freeze,EM,F1
2,squadid,True,True,50.40963,67.63862
1,squadid,True,False,49.94148,67.20374
0,squadid,Baseline,Baseline,49.54021,66.69283


# Inference with NLI validation

In [34]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

In [35]:
def take_all_url_nli_validator(data):

    baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix}"
    msc_arr = ["indonli", "indonli_mnli", f"indonli_mnli_{data}-nli"]
    tq_arr = [1, 2]
    msi_arr = [1, 2, 3]
    var_arr = [1, 2, 3]
    th_arr = [0.25, 0.5, 0.75]

    url_arr = []
    url_arr.append(baseline)

    for msc in msc_arr:
        for tq in tq_arr:
            for msi in msi_arr:
                for var in var_arr:
                    for th in th_arr:
                        
                        if msc == "indonli" or msc == "indonli_mnli":
                            var = 1
                            th = 0.0

                        if var == 1:
                            th = 0.0

                        url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [36]:
def extract_values_nli_validator(url, data):

    msc_list = f"indonli_mnli_{data}-nli|indonli_mnli|indonli"
    pattern = re.compile(r'.*FilteringNLI-({})-{}-TQ(\d+)-TS(\d+)-MS([\d]+)-VA([\d]+)-TH([\d.]+).*'.format(msc_list, data))
    match = pattern.match(url)
    
    if match:
        msc = match.group(1)
        TQ = int(match.group(2))
        TS = int(match.group(3))
        MSI = int(match.group(4))
        VA = int(match.group(5))
        
        TH = float(match.group(6))
        if TH == '0.00':
            TH = '0.0'
        elif TH == '0.50':
            TH = '0.5'

        EM = em_and_f1(url, True)
        F1 = em_and_f1(url, False)

        return msc, data, TQ, TS, MSI, VA, TH, EM, F1
    
    else:
        return None

In [40]:
def create_table_nli_validator(url_list, data):
    
    data_list = []
    for url in tqdm(url_list, desc="Processing URLs", unit="URL"):
        extracted_values = extract_values_nli_validator(url, data)
        data_list.append(extracted_values)

    df = pd.DataFrame(data_list, columns=['MSC', 'data', 'TQ', 'TS', 'MSI', 'VA', 'TH', 'EM', 'F1'])
    
    df.loc[0, 'MSC'] = "Baseline"
    df.loc[0, 'TQ'] = "BL"
    df.loc[0, 'TS'] = "BL"
    df.loc[0, 'MSI'] = "BL"
    df.loc[0, 'VA'] = "BL"
    df.loc[0, 'TH'] = "BL"
    
    return df

In [41]:
data = "idkmrc"
url_list = take_all_url_nli_validator(data)
table = pd.DataFrame(create_table_nli_validator(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 55


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 55/55 [00:47<00:00,  1.17URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.82938,84.25128
1,indonli,idkmrc,1,4,1,1,0.0,68.48341,75.02483
2,indonli,idkmrc,1,4,2,1,0.0,54.62085,62.08946
3,indonli,idkmrc,1,4,3,1,0.0,43.95735,51.95651
4,indonli,idkmrc,2,4,1,1,0.0,75.35545,83.42190
5,indonli,idkmrc,2,4,2,1,0.0,73.93365,82.22194
6,indonli,idkmrc,2,4,3,1,0.0,72.63033,80.93617
7,indonli_mnli,idkmrc,1,4,1,1,0.0,71.32701,78.02263
8,indonli_mnli,idkmrc,1,4,2,1,0.0,61.96682,69.50875
9,indonli_mnli,idkmrc,1,4,3,1,0.0,52.60664,60.54560


In [42]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

idkmrc sorted by EM


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
37,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,2,0.75,75.94787,84.16834
16,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,2,0.75,75.94787,84.16834
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.82938,84.25128
13,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,1,0.0,75.82938,84.16834
36,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,2,0.5,75.82938,84.16834
35,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,2,0.25,75.82938,84.16834
34,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,1,0.0,75.82938,84.16834
40,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,3,0.75,75.82938,84.25128
39,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,3,0.5,75.82938,84.25128
18,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,3,0.5,75.82938,84.25128


In [43]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

idkmrc sorted by F1


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.82938,84.25128
17,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,3,0.25,75.82938,84.25128
40,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,3,0.75,75.82938,84.25128
39,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,3,0.5,75.82938,84.25128
18,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,3,0.5,75.82938,84.25128
19,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,3,0.75,75.82938,84.25128
38,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,3,0.25,75.82938,84.25128
34,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,1,0.0,75.82938,84.16834
37,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,2,0.75,75.94787,84.16834
36,indonli_mnli_idkmrc-nli,idkmrc,2,4,1,2,0.5,75.82938,84.16834


## TYDI-QA-ID

In [44]:
data = "tydiqaid"
url_list = take_all_url_nli_validator(data)
table = pd.DataFrame(create_table_nli_validator(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 55


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 55/55 [00:47<00:00,  1.15URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,tydiqaid,BL,BL,BL,BL,BL,61.28655,76.47617
1,indonli,tydiqaid,1,4,1,1,0.0,35.90643,46.40946
2,indonli,tydiqaid,1,4,2,1,0.0,36.14035,48.78837
3,indonli,tydiqaid,1,4,3,1,0.0,36.25731,50.22544
4,indonli,tydiqaid,2,4,1,1,0.0,59.76608,74.66427
5,indonli,tydiqaid,2,4,2,1,0.0,60.00000,75.23357
6,indonli,tydiqaid,2,4,3,1,0.0,60.11696,75.35280
7,indonli_mnli,tydiqaid,1,4,1,1,0.0,42.92398,54.29436
8,indonli_mnli,tydiqaid,1,4,2,1,0.0,43.50877,56.56735
9,indonli_mnli,tydiqaid,1,4,3,1,0.0,43.50877,57.57900


In [45]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

tydiqaid sorted by EM


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
0,Baseline,tydiqaid,BL,BL,BL,BL,BL,61.28655,76.47617
41,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,1,0.0,61.28655,77.30986
25,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.5,61.28655,77.45290
28,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,2,0.25,61.28655,77.30986
29,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,2,0.5,61.28655,77.30986
31,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,3,0.25,61.28655,77.33594
32,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,3,0.5,61.28655,77.33594
38,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,1,3,0.25,61.28655,76.47617
39,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,1,3,0.5,61.28655,76.47617
40,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,1,3,0.75,61.28655,76.47617


In [46]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

tydiqaid sorted by F1


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1
25,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.5,61.28655,77.45290
46,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,3,0.5,61.28655,77.45290
45,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,3,0.25,61.28655,77.45290
24,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.25,61.28655,77.45290
47,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,2,3,0.75,61.16959,77.42606
26,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.75,61.16959,77.42606
31,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,3,0.25,61.28655,77.33594
53,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,3,3,0.5,61.28655,77.33594
52,indonli_mnli_tydiqaid-nli,tydiqaid,2,4,3,3,0.25,61.28655,77.33594
32,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,3,3,0.5,61.28655,77.33594


## SQUAD-ID

In [ ]:
data = "squadid"
url_list = take_all_url_nli_validator(data)
table = pd.DataFrame(create_table_nli_validator(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 55


Processing URLs:  42%|███████████████████████████▏                                     | 23/55 [00:20<00:27,  1.18URL/s]

In [ ]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

In [ ]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table